# W_LEDGER_DS ETL Process
### Oracle EBS GL_LEDGERS to EDW W_LEDGER_DS

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  'Y' AS IS_INCREMENTAL,
  -- Derive last extract date from control table; fallback to 1970-01-01 if no record found
  COALESCE(
    date_format(max(ETL_LOAD_DATE), 'yyyy-MM-dd HH:mm:ss'),
    '1970-01-01 00:00:00'
  ) AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2624261 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS,
  'US' AS LANGUAGE_BASE
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__';

In [ ]:
%sql
-- 2) STAGING: Read source tables and apply transformations
CREATE OR REPLACE TEMP VIEW stg_gl_ledgers AS
SELECT
  gl.NAME                                  AS name,
  gl.SHORT_NAME                            AS short_name,
  gl.DESCRIPTION                           AS description,
  gl.PERIOD_SET_NAME                       AS period_set_name,
  gl.LAST_UPDATE_DATE                      AS last_update_date,
  gl.CREATION_DATE                         AS creation_date,
  gpt.USER_PERIOD_TYPE                     AS user_period_type,
  CAST(gl.CHART_OF_ACCOUNTS_ID AS STRING)  AS chart_of_accounts,
  COALESCE(gl.CURRENCY_CODE, '__UNASSIGNED__') AS currency_code,
  COALESCE(gl.SLA_ACCOUNTING_METHOD_CODE, '__UNASSIGNED__') AS sla_accounting_method_code,
  COALESCE(gl.LEDGER_CATEGORY_CODE, '__UNASSIGNED__') AS ledger_category_code,
  CAST(gl.CREATED_BY AS STRING)            AS created_by_id,
  CAST(gl.LAST_UPDATED_BY AS STRING)       AS changed_by_id,
  CAST(gl.LEDGER_ID AS STRING)             AS integration_id
FROM workspace.oracle_ebs.GL_LEDGERS gl
INNER JOIN workspace.oracle_ebs.GL_PERIOD_TYPES gpt
  ON gl.ACCOUNTED_PERIOD_TYPE = gpt.PERIOD_TYPE
LEFT JOIN workspace.oracle_ebs.XLA_ACCTG_METHODS_TL xam
  ON gl.SLA_ACCOUNTING_METHOD_TYPE = xam.ACCOUNTING_METHOD_TYPE_CODE
  AND gl.SLA_ACCOUNTING_METHOD_CODE = xam.ACCOUNTING_METHOD_CODE
  AND xam.LANGUAGE = (SELECT LANGUAGE_BASE FROM etl_params)
CROSS JOIN etl_params p
WHERE COALESCE(gl.COMPLETE_FLAG, 'Y') = 'Y'
  AND gl.OBJECT_TYPE_CODE = 'L';

In [ ]:
%sql
-- 3) INSERT: Load data into target table W_LEDGER_DS
INSERT INTO workspace.oracle_edw.w_ledger_ds
(
  LEDGER_NAME,
  LEDGER_SHORT_NAME,
  LEDGER_DESC,
  CHART_OF_ACCOUNTS,
  CURRENCY_CODE,
  CALENDER_NAME,
  SLA_ACCOUNTING_METHOD_CODE,
  PERIOD_TYPE,
  LEDGER_CATEGORY_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  DELETE_FLG,
  TENANT_ID
)
SELECT
  s.name                    AS LEDGER_NAME,
  s.short_name              AS LEDGER_SHORT_NAME,
  s.description             AS LEDGER_DESC,
  s.chart_of_accounts       AS CHART_OF_ACCOUNTS,
  s.currency_code           AS CURRENCY_CODE,
  s.period_set_name         AS CALENDER_NAME,
  s.sla_accounting_method_code AS SLA_ACCOUNTING_METHOD_CODE,
  s.user_period_type        AS PERIOD_TYPE,
  s.ledger_category_code    AS LEDGER_CATEGORY_CODE,
  s.created_by_id           AS CREATED_BY_ID,
  s.changed_by_id           AS CHANGED_BY_ID,
  try_cast(s.creation_date AS TIMESTAMP) AS CREATED_ON_DT,
  try_cast(s.last_update_date AS TIMESTAMP) AS CHANGED_ON_DT,
  p.DATASOURCE_NUM_ID       AS DATASOURCE_NUM_ID,
  s.integration_id          AS INTEGRATION_ID,
  'N'                       AS DELETE_FLG,
  p.TENANT_ID               AS TENANT_ID
FROM stg_gl_ledgers s
CROSS JOIN etl_params p;

In [ ]:
%sql
-- 4) UPDATE CONTROL TABLE: Upsert W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
    p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION' AS package_name,
    'W_LEDGER_DS' AS target_table_name,
    p.ETL_USAGE_CODE    AS etl_usage_code,
    p.ETL_PROC_WID      AS etl_proc_wid,
    p.EXECUTION_ID      AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN
  UPDATE SET
    tgt.target_table_name = src.target_table_name,
    tgt.etl_proc_wid = src.etl_proc_wid,
    tgt.load_plan_id = src.load_plan_id,
    tgt.wip_load_start_date = src.wip_load_start_date,
    tgt.etl_load_date = src.etl_load_date,
    tgt.committed = src.committed
WHEN NOT MATCHED THEN
  INSERT (
    datasource_num_id, package_name, target_table_name, etl_usage_code,
    etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
  )
  VALUES (
    src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
    src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed
  );

In [ ]:
%sql
-- 5) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  p.DATASOURCE_NUM_ID,
  'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION',
  'W_LEDGER_DS',
  p.ETL_USAGE_CODE,
  p.ETL_PROC_WID,
  p.EXECUTION_ID,
  p.ETL_PROC_WID AS SESSION_ID,
  date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
  NULL AS LAST_MAX_DATE,
  current_timestamp() AS ETL_LOAD_DATE,
  CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
FROM etl_params p;

In [ ]:
%sql
-- 6) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS stg_gl_ledgers;
DROP VIEW IF EXISTS etl_params;